In [4]:
import sys

sys.path.insert(0, '../Lab/')
sys.path.insert(0, '../New_Hybrids/')
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from myRecommenders.UserIcmKNNCFRecommender import UserIcmKNNCFRecommender
from myRecommenders.ItemIcmKNNCFRecommender import ItemIcmKNNCFRecommender
from myRecommenders.WeightedHybrid import WeightedHybrid
from myRecommenders.WeightedHybridV2 import WeightedHybridScoreRecommender
from myRecommenders.WeightedListHybrid import WeightedListHybrid
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from GraphBased.P3alphaRecommender import P3alphaRecommender

import matplotlib
import matplotlib.pyplot as plt
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python

%matplotlib inline 

import numpy as np

In [5]:
sys.path.insert(0, '../datasets/')
from load_tables import load_ICM_csr, load_URM_csr
ICM_all = load_ICM_csr()
URM_all = load_URM_csr()

## Recommenders:
recs = [
    P3alphaRecommender,
    ItemKNNCFRecommender,
    UserKNNCFRecommender,
    UserIcmKNNCFRecommender,
    RP3betaRecommender,
    ItemIcmKNNCFRecommender,
    IALSRecommender 
    ]

## Alcune riflessioni
- Non usare BPR pare funzioni bene su non scaled , xk bpr ha pesi un po' crazy
- 7 recommenders anche se tanti non male
- Itme ICM pare funzioni benino in combo con altri

## Baseline

In [6]:
from Notebooks_utils.evaluation_function import evaluate_algorithm
from Notebooks_utils.data_splitter import train_test_holdout
from Base.Evaluation.Evaluator import EvaluatorHoldout

URM_train, URM_test = train_test_holdout(URM_all, train_perc = 0.8)

evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

## Tuning

In [3]:
from Notebooks_utils.evaluation_function import evaluate_algorithm
from Notebooks_utils.data_splitter import train_test_holdout
from Base.Evaluation.Evaluator import EvaluatorHoldout

URM_train_validation, URM_test = train_test_holdout(URM_all, train_perc = 0.8)
URM_train, URM_validation = train_test_holdout(URM_train_validation, train_perc = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

from skopt.space import Real, Integer, Categorical
from ParameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 500
n_random_starts = int(n_cases*0.12)
metric_to_optimize = "MAP"

In [4]:
hyperparameters_range_dictionary = {}#forse qui fa il matto
hyperparameters_range_dictionary["w0"] = Real(0.0001,1) 
hyperparameters_range_dictionary["w1"] = Real(0.0001,1) 
hyperparameters_range_dictionary["w2"] = Real(0.0001,1) 
hyperparameters_range_dictionary["w3"] = Real(0.0001,1) 
hyperparameters_range_dictionary["w4"] = Real(0.0001,1) 
hyperparameters_range_dictionary["w5"] = Real(0.0001,1) 
hyperparameters_range_dictionary["w6"] = Real(0.0001,1) 
#hyperparameters_range_dictionary["w7"] = Real(0.0001,1)

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train,ICM_all],#forse qui bugga va tolta icm      
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)


recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation,ICM_all],    
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

 

In [5]:
from myRecommenders.WeightedHybridV2forBayesianSearch import WeightedHybridV2forBayesianSearch
recommender_class=WeightedHybridV2forBayesianSearch

In [6]:

import time
start_time = time.time()
parameterSearch = SearchBayesianSkopt(recommender_class,
                                 evaluator_validation=evaluator_validation,
                                 evaluator_test=evaluator_test)


parameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       parameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path,
                       output_file_name_root = recommender_class.RECOMMENDER_NAME,
                       metric_to_optimize = metric_to_optimize
                      )

end_time = time.time()

print("Ha finito in questo tempo: "+str(end_time-start_time)+" secondi")

Iteration No: 1 started. Evaluating function at random point.
WeightedHybridV2forBayesianSearch: URM Detected 222 (2.79 %) cold users.
WeightedHybridV2forBayesianSearch: URM Detected 4034 (15.53 %) cold items.
TopPopRecommender: URM Detected 222 (2.79 %) cold users.
TopPopRecommender: URM Detected 4034 (15.53 %) cold items.
P3alphaRecommender: URM Detected 222 (2.79 %) cold users.
P3alphaRecommender: URM Detected 4034 (15.53 %) cold items.
ItemKNNCFRecommender: URM Detected 222 (2.79 %) cold users.
ItemKNNCFRecommender: URM Detected 4034 (15.53 %) cold items.
UserKNNCFRecommender: URM Detected 222 (2.79 %) cold users.
UserKNNCFRecommender: URM Detected 4034 (15.53 %) cold items.
UserKNNCFRecommender: URM Detected 222 (2.79 %) cold users.
UserKNNCFRecommender: URM Detected 4034 (15.53 %) cold items.
UserKNNCFRecommender: URM Detected 223 (2.81 %) cold users.
UserKNNCFRecommender: URM Detected 102 (0.51 %) cold items.
RP3betaRecommender: URM Detected 222 (2.79 %) cold users.
RP3betaRecom

Similarity column 25975 ( 100 % ), 8705.08 column/sec, elapsed time 0.05 min
Similarity column 7947 ( 100 % ), 21192.67 column/sec, elapsed time 0.01 min
Similarity column 7947 ( 100 % ), 20450.00 column/sec, elapsed time 0.01 min
Similarity column 7947 ( 100 % ), 3586.60 column/sec, elapsed time 0.04 min
Similarity column 25975 ( 100 % ), 8540.11 column/sec, elapsed time 0.05 min
Similarity column 25975 ( 100 % ), 4290.99 column/sec, elapsed time 0.10 min
IALSRecommender: Epoch 1 of 45. Elapsed time 3.36 sec
IALSRecommender: Epoch 2 of 45. Elapsed time 6.80 sec
IALSRecommender: Epoch 3 of 45. Elapsed time 10.30 sec
IALSRecommender: Epoch 4 of 45. Elapsed time 13.66 sec
IALSRecommender: Epoch 5 of 45. Elapsed time 17.08 sec
IALSRecommender: Epoch 6 of 45. Elapsed time 20.41 sec
IALSRecommender: Epoch 7 of 45. Elapsed time 23.75 sec
IALSRecommender: Epoch 8 of 45. Elapsed time 27.15 sec
IALSRecommender: Epoch 9 of 45. Elapsed time 30.64 sec
IALSRecommender: Epoch 10 of 45. Elapsed time 

IALSRecommender: Epoch 23 of 45. Elapsed time 1.46 min
IALSRecommender: Epoch 24 of 45. Elapsed time 1.52 min
IALSRecommender: Epoch 25 of 45. Elapsed time 1.59 min
IALSRecommender: Epoch 26 of 45. Elapsed time 1.65 min
IALSRecommender: Epoch 27 of 45. Elapsed time 1.71 min
IALSRecommender: Epoch 28 of 45. Elapsed time 1.77 min
IALSRecommender: Epoch 29 of 45. Elapsed time 1.82 min
IALSRecommender: Epoch 30 of 45. Elapsed time 1.88 min
IALSRecommender: Epoch 31 of 45. Elapsed time 1.94 min
IALSRecommender: Epoch 32 of 45. Elapsed time 2.00 min
IALSRecommender: Epoch 33 of 45. Elapsed time 2.06 min
IALSRecommender: Epoch 34 of 45. Elapsed time 2.12 min
IALSRecommender: Epoch 35 of 45. Elapsed time 2.17 min
IALSRecommender: Epoch 36 of 45. Elapsed time 2.23 min
IALSRecommender: Epoch 37 of 45. Elapsed time 2.28 min
IALSRecommender: Epoch 38 of 45. Elapsed time 2.34 min
IALSRecommender: Epoch 39 of 45. Elapsed time 2.40 min
IALSRecommender: Epoch 40 of 45. Elapsed time 2.46 min
IALSRecomm

UserKNNCFRecommender: URM Detected 223 (2.81 %) cold users.
UserKNNCFRecommender: URM Detected 102 (0.51 %) cold items.
RP3betaRecommender: URM Detected 222 (2.79 %) cold users.
RP3betaRecommender: URM Detected 4034 (15.53 %) cold items.
ItemKNNCFRecommender: URM Detected 222 (2.79 %) cold users.
ItemKNNCFRecommender: URM Detected 4034 (15.53 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
IALSRecommender: URM Detected 222 (2.79 %) cold users.
IALSRecommender: URM Detected 4034 (15.53 %) cold items.
SearchBayesianSkopt: Testing config: {'w0': 0.6914532169141507, 'w1': 0.9068798802155333, 'w2': 0.7914165426325788, 'w3': 0.09465416062762569, 'w4': 0.845073041270033, 'w5': 0.7094009587124726, 'w6': 0.8188521235006117}
--------FITTING IN PROGRESS...-------
Similarity column 25975 ( 100 % ), 8373.41 column/sec, elapsed time 0.05 min
Similarity column 7947 ( 100 % ), 20500.07 column/sec, elapsed time 0.01 min

IALSRecommender: Epoch 24 of 45. Elapsed time 1.65 min
IALSRecommender: Epoch 25 of 45. Elapsed time 1.71 min
IALSRecommender: Epoch 26 of 45. Elapsed time 1.78 min
IALSRecommender: Epoch 27 of 45. Elapsed time 1.85 min
IALSRecommender: Epoch 28 of 45. Elapsed time 1.92 min
IALSRecommender: Epoch 29 of 45. Elapsed time 1.99 min
IALSRecommender: Epoch 30 of 45. Elapsed time 2.06 min
IALSRecommender: Epoch 31 of 45. Elapsed time 2.13 min
IALSRecommender: Epoch 32 of 45. Elapsed time 2.20 min
IALSRecommender: Epoch 33 of 45. Elapsed time 2.27 min
IALSRecommender: Epoch 34 of 45. Elapsed time 2.34 min
IALSRecommender: Epoch 35 of 45. Elapsed time 2.41 min
IALSRecommender: Epoch 36 of 45. Elapsed time 2.48 min
IALSRecommender: Epoch 37 of 45. Elapsed time 2.55 min
IALSRecommender: Epoch 38 of 45. Elapsed time 2.62 min
IALSRecommender: Epoch 39 of 45. Elapsed time 2.69 min
IALSRecommender: Epoch 40 of 45. Elapsed time 2.76 min
IALSRecommender: Epoch 41 of 45. Elapsed time 2.83 min
IALSRecomm

...

IALSRecommender: Epoch 34 of 45. Elapsed time 2.54 min
IALSRecommender: Epoch 35 of 45. Elapsed time 2.63 min
IALSRecommender: Epoch 36 of 45. Elapsed time 2.72 min
IALSRecommender: Epoch 37 of 45. Elapsed time 2.80 min
IALSRecommender: Epoch 38 of 45. Elapsed time 2.89 min
IALSRecommender: Epoch 39 of 45. Elapsed time 2.98 min
IALSRecommender: Epoch 40 of 45. Elapsed time 3.06 min
IALSRecommender: Epoch 41 of 45. Elapsed time 3.15 min
IALSRecommender: Epoch 42 of 45. Elapsed time 3.24 min
IALSRecommender: Epoch 43 of 45. Elapsed time 3.33 min
IALSRecommender: Epoch 44 of 45. Elapsed time 3.42 min
IALSRecommender: Epoch 45 of 45. Elapsed time 3.51 min
IALSRecommender: Terminating at epoch 45. Elapsed time 3.51 min
------FITTING END, SIAMO GROSSISSIMI ------
EvaluatorHoldout: Processed 5175 ( 100.00% ) in 23.42 sec. Users per second: 221
SearchBayesianSkopt: Config 483 is suboptimal. Config: {'w0': 0.8147738224239099, 'w1': 0.0001, 'w2': 0.2990145984285526, 'w3': 0.14082010674819206, 'w

ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
IALSRecommender: URM Detected 222 (2.79 %) cold users.
IALSRecommender: URM Detected 4034 (15.53 %) cold items.
SearchBayesianSkopt: Testing config: {'w0': 0.6758551427708873, 'w1': 0.2144509306788503, 'w2': 0.4112219051523297, 'w3': 0.10549779943692475, 'w4': 0.16496066799744122, 'w5': 1.0, 'w6': 0.8054050101734245}
--------FITTING IN PROGRESS...-------
Similarity column 25975 ( 100 % ), 7101.12 column/sec, elapsed time 0.06 min
Similarity column 7947 ( 100 % ), 16460.88 column/sec, elapsed time 0.01 min
Similarity column 7947 ( 100 % ), 18276.98 column/sec, elapsed time 0.01 min
Similarity column 7947 ( 100 % ), 2870.53 column/sec, elapsed time 0.05 min
Similarity column 25975 ( 100 % ), 7513.07 column/sec, elapsed time 0.06 min
Similarity column 25975 ( 100 % ), 3763.52 column/sec, elapsed time 0.12 min
IALSRecommender: Epoch 1 of 45. Elapsed time 5.25 sec
IALSRecommender: Epoch 2 of 45. Elapsed time 11.05 sec
IALSRecommende

IALSRecommender: Epoch 31 of 45. Elapsed time 2.80 min
IALSRecommender: Epoch 32 of 45. Elapsed time 2.89 min
IALSRecommender: Epoch 33 of 45. Elapsed time 2.98 min
IALSRecommender: Epoch 34 of 45. Elapsed time 3.07 min
IALSRecommender: Epoch 35 of 45. Elapsed time 3.16 min
IALSRecommender: Epoch 36 of 45. Elapsed time 3.25 min
IALSRecommender: Epoch 37 of 45. Elapsed time 3.34 min
IALSRecommender: Epoch 38 of 45. Elapsed time 3.43 min
IALSRecommender: Epoch 39 of 45. Elapsed time 3.52 min
IALSRecommender: Epoch 40 of 45. Elapsed time 3.62 min
IALSRecommender: Epoch 41 of 45. Elapsed time 3.71 min
IALSRecommender: Epoch 42 of 45. Elapsed time 3.80 min
IALSRecommender: Epoch 43 of 45. Elapsed time 3.89 min
IALSRecommender: Epoch 44 of 45. Elapsed time 3.98 min
IALSRecommender: Epoch 45 of 45. Elapsed time 4.07 min
IALSRecommender: Terminating at epoch 45. Elapsed time 4.07 min
------FITTING END, SIAMO GROSSISSIMI ------
EvaluatorHoldout: Processed 5175 ( 100.00% ) in 23.89 sec. Users pe

Similarity column 25975 ( 100 % ), 7247.86 column/sec, elapsed time 0.06 min
Similarity column 7947 ( 100 % ), 15505.99 column/sec, elapsed time 0.01 min
Similarity column 7947 ( 100 % ), 18479.28 column/sec, elapsed time 0.01 min
Similarity column 7947 ( 100 % ), 3335.88 column/sec, elapsed time 0.04 min
Similarity column 25975 ( 100 % ), 7581.23 column/sec, elapsed time 0.06 min
Similarity column 25975 ( 100 % ), 3581.38 column/sec, elapsed time 0.12 min
IALSRecommender: Epoch 1 of 45. Elapsed time 4.72 sec


KeyboardInterrupt: 

In [9]:
from Base.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

best_parameters = search_metadata["hyperparameters_best"]
#settaggio manuale: BESTPARAMS={'w0': 0.7890828279804322, 'w1': 0.0001, 'w2': 0.31527729065553595, 'w3': 0.12162462193350662, 'w4': 0.06519442814723547, 'w5': 1.0, 'w6': 0.3445924625940507}
print(best_parameters)

#è[OLD]best con 7 recc={'w0': 0.001, 'w1': 0.5593506236737317, 'w2': 0.23267769028384777, 'w3': 0.10077656126622785, 'w4': 0.0001, 'w5': 0.0001, 'w6': 0.8397993296794579}

{'w0': 0.7890828279804322, 'w1': 0.0001, 'w2': 0.31527729065553595, 'w3': 0.12162462193350662, 'w4': 0.06519442814723547, 'w5': 1.0, 'w6': 0.3445924625940507}


In [10]:
URM_train, URM_test = train_test_holdout(URM_all, train_perc = 0.8)

evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

from myRecommenders.WeightedHybridV2forBayesianSearch import WeightedHybridV2forBayesianSearch
recommender=WeightedHybridV2forBayesianSearch(URM_train,ICM_all)
#IL SECONDO E' IL VERO BEST, 
#configuration: recs = [ P3alphaRecommender, ItemKNNCFRecommender, UserKNNCFRecommender, UserIcmKNNCFRecommender, RP3betaRecommender, ItemIcmKNNCFRecommender, IALSRecommender ]
#best_parameters={'w0': 0.24747685546888099, 'w1': 0.16035036296079344, 'w2': 0.49965929661862485, 'w3': 0.10566910974168996, 'w4': 0.0001, 'w5': 0.9774213225381028, 'w6': 1.0}
#MAP=0.068 bad
best_parameters={'w0': 0.7890828279804322, 'w1': 0.0001, 'w2': 0.31527729065553595, 'w3': 0.12162462193350662, 'w4': 0.06519442814723547, 'w5': 1.0, 'w6': 0.3445924625940507}
#MAP=0.07282438619685525

recommender.fit(**best_parameters)

WeightedHybridV2forBayesianSearch: URM Detected 58 (0.73 %) cold users.
WeightedHybridV2forBayesianSearch: URM Detected 2438 (9.39 %) cold items.
TopPopRecommender: URM Detected 58 (0.73 %) cold users.
TopPopRecommender: URM Detected 2438 (9.39 %) cold items.
P3alphaRecommender: URM Detected 58 (0.73 %) cold users.
P3alphaRecommender: URM Detected 2438 (9.39 %) cold items.
ItemKNNCFRecommender: URM Detected 58 (0.73 %) cold users.
ItemKNNCFRecommender: URM Detected 2438 (9.39 %) cold items.
UserKNNCFRecommender: URM Detected 58 (0.73 %) cold users.
UserKNNCFRecommender: URM Detected 2438 (9.39 %) cold items.
UserKNNCFRecommender: URM Detected 58 (0.73 %) cold users.
UserKNNCFRecommender: URM Detected 2438 (9.39 %) cold items.
UserKNNCFRecommender: URM Detected 58 (0.73 %) cold users.
UserKNNCFRecommender: URM Detected 37 (0.18 %) cold items.
RP3betaRecommender: URM Detected 58 (0.73 %) cold users.
RP3betaRecommender: URM Detected 2438 (9.39 %) cold items.
ItemKNNCFRecommender: URM Dete

In [11]:
result_dict, _ = evaluator_test.evaluateRecommender(recommender)
MAP = result_dict[10]["MAP"]
print("MAP  recommeder: "+str(MAP)+"\n")#0.0715

EvaluatorHoldout: Processed 5588 ( 100.00% ) in 24.60 sec. Users per second: 227
MAP  recommeder: 0.07282438619685525



# Scrivi consegna

In [12]:
#---7 parameters
recommender=WeightedHybridV2forBayesianSearch(URM_all,ICM_all)
#best_parameters={'w0': 0.24747685546888099, 'w1': 0.16035036296079344, 'w2': 0.49965929661862485, 'w3': 0.10566910974168996, 'w4': 0.0001, 'w5': 0.9774213225381028, 'w6': 1.0}
best_parameters={'w0': 0.7890828279804322, 'w1': 0.0001, 'w2': 0.31527729065553595, 'w3': 0.12162462193350662, 'w4': 0.06519442814723547, 'w5': 1.0, 'w6': 0.3445924625940507}
recommender.fit(**best_parameters)

WeightedHybridV2forBayesianSearch: URM Detected 1079 (4.15 %) cold items.
TopPopRecommender: URM Detected 1079 (4.15 %) cold items.
P3alphaRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNCFRecommender: URM Detected 1079 (4.15 %) cold items.
UserKNNCFRecommender: URM Detected 1079 (4.15 %) cold items.
UserKNNCFRecommender: URM Detected 1079 (4.15 %) cold items.
UserKNNCFRecommender: URM Detected 14 (0.07 %) cold items.
RP3betaRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNCFRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
IALSRecommender: URM Detected 1079 (4.15 %) cold items.
--------FITTING IN PROGRESS...-------
Similarity column 25975 ( 100 % ), 6505.03 column/sec, elapsed time 0.07 min
Similarity column 7947 ( 100 % ), 12878.02 column/sec, elapsed time 0.01 min
Similarity column 7947 ( 100 % ), 12866.27 column/sec, elapsed time 0.01 min
Simil

In [13]:
import time
URM_file = open(r"C:\Users\aless\PYTHON_NOTEBOOKS\RECOMMENDER_CHALLENGE\data_target_users_test.csv", 'r')

URM_file.seek(0)
URM_tuples = []
URM_file.readline()#questa è importante di cristo c'è riga di introduzione
for line in URM_file:
    URM_tuples.append(int(line.replace("\n","")))

usersToPred = np.array(URM_tuples)

start_time = time.time()#BUONE METRICHE PER IL TEMPO QUI
import csv

with open('OUTPUTS\outputIbridoEnrico.csv', mode='w',newline='') as recFile:
    
    recFile = csv.writer(recFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    recFile.writerow(['user_id', 'item_list'])
    recs=recommender.recommend(usersToPred, cutoff=10)
    for user_id,rec in zip(usersToPred,recs):
        rec_string =' '.join(map(str, rec[:10]))
        recFile.writerow([user_id, rec_string])
    
end_time = time.time()
print("Ha finito in questo tempo: "+str(end_time-start_time)+" secondi")

Ha finito in questo tempo: 32.833516120910645 secondi
